In [1]:
import os
import string
import annoy
import codecs

from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

import numpy as np
from tqdm.notebook import tqdm
import pandas as pd
import json
from sklearn.model_selection import train_test_split
import bz2file as bz2
import pickle

In [2]:
morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)

def preprocess_txt(line: str):
    """Функция, предобрабатывающая текст. На вход принимает строку текста, возвращает список обработаных слов"""
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

## Подгрузка имеющихся моделей и индексов
В ячейке ниже задокументирован блок кода из прошлого задания, в котором обучена модель на вопросах из mail.ru, создан annoy индекс по вопросам и словарь с ответами по индексам. Всё сохранено в файлы. Далее в функциях будут использованы загруженные из этих файлов модели и индексы.

In [3]:
question = None
written = False

#Мы идем по всем записям, берем первую строку как вопрос
# и после знака --- находим ответ
with codecs.open("prepared_answers.txt","w", "utf-8") as fout:
    with codecs.open("Otvety.txt", "r", "utf-8") as fin:
        for line in tqdm(fin):
            if line.startswith("---"):
                written = False
                continue
            if not written and question is not None:
                fout.write(question.replace("\t", " ").strip() + "\t" + line.replace("\t", " "))
                written = True
                question = None
                continue
            if not written:
                question = line.strip()
                continue

0it [00:00, ?it/s]

In [26]:
sentences = []
c = 0

with codecs.open("prepared_answers.txt", "r", "utf-8") as fin:
    for line in tqdm(fin):
        spls = preprocess_txt(line)
        sentences.append(spls)
        c += 1
        if c > 300000:
            break

0it [00:00, ?it/s]

In [27]:
# Обучим модель word2vec на наших вопросах
sentences = [i for i in sentences if len(i) > 2]
model_speaker = Word2Vec(sentences=sentences, vector_size=50, min_count=3, window=5)
model_speaker.save("w2v_model")

In [28]:
index = annoy.AnnoyIndex(50 ,'angular')

index_map = {}
counter = 0

with codecs.open("prepared_answers.txt", "r", "utf-8") as f:
    for line in tqdm(f):
        n_w2v = 0
        spls = line.split("\t")
        index_map[counter] = spls[1]
        question = preprocess_txt(spls[0])
        vector = np.zeros(50)
        for word in question:
            if word in model_speaker.wv:
                vector += model_speaker.wv[word]
                n_w2v += 1
        if n_w2v > 0:
            vector = vector / n_w2v
        index.add_item(counter, vector)
            
        counter += 1
        if counter>300000:
            break

index.build(6)
index.save('speaker.ann')

0it [00:00, ?it/s]

True

In [29]:
# Сохраним словарь с ответами в файл
with open('dict_for_speaker.pkl', 'wb') as file:
     pickle.dump(index_map, file)

In [5]:
question = None
written = False

#Мы идем по всем записям, берем первую строку как вопрос
# и после знака --- находим ответ
with codecs.open("prepared_answers.txt","w", "utf-8") as fout:
    with codecs.open("Otvety.txt", "r", "utf-8") as fin:
        for line in tqdm(fin):
            if line.startswith("---"):
                written = False
                continue
            if not written and question is not None:
                fout.write(question.replace("\t", " ").strip() + "\t" + line.replace("\t", " "))
                written = True
                question = None
                continue
            if not written:
                question = line.strip()
                continue
                
sentences = []
c = 0

with codecs.open("prepared_answers.txt", "r", "utf-8") as fin:
    for line in tqdm(fin):
        spls = preprocess_txt(line)
        sentences.append(spls)
        c += 1
        if c > 500000:
            break
            
# Обучим модель word2vec на наших вопросах
sentences = [i for i in sentences if len(i) > 2]
model = Word2Vec(sentences=sentences, vector_size=100, min_count=1, window=5)
model.save("w2v_model")

index = annoy.AnnoyIndex(100 ,'angular')

index_map = {}
counter = 0

with codecs.open("prepared_answers.txt", "r", "utf-8") as f:
    for line in tqdm(f):
        n_w2v = 0
        spls = line.split("\t")
        index_map[counter] = spls[1]
        question = preprocess_txt(spls[0])
        vector = np.zeros(100)
        for word in question:
            if word in model_speaker.wv:
                vector += model_speaker.wv[word]
                n_w2v += 1
        if n_w2v > 0:
            vector = vector / n_w2v
        index.add_item(counter, vector)
            
        counter += 1

index.build(10)
index.save('speaker.ann')

# Сохраним словарь с ответами в файл
with open('dict_for_speaker.pkl', 'wb') as file:
     pickle.dump(index_map, file)

In [3]:
# Загружаем из файлов словарь для чат-бота, модель и индексы. Будем их использовать в функции speaker_answer
index_map = pickle.load(open('dict_for_speaker.pkl', 'rb'))
#model_speaker = KeyedVectors.load('w2v_model')
index_speaker = annoy.AnnoyIndex(100 ,'angular')
index_speaker.load('speaker.ann')

True

In [31]:
model_speaker = Word2Vec.load('w2v_model')

In [7]:
# Pickle a file and then compress it into a file with extension 
def compressed_pickle(title, data):
    with bz2.BZ2File(title + '.pbz2', 'wb') as f: 
        pickle.dump(data, f, protocol=5)
    
    
# Load any compressed pickle file
def decompress_pickle(file):
    data = bz2.BZ2File(file, 'rb')
    data = pickle.load(data)
    return data

In [56]:
index_speaker = annoy.AnnoyIndex(50 ,'angular')
index_speaker.load('speaker.ann')

True

In [16]:
import zipfile

In [30]:
compressed_pickle('dict_for_speaker.pkl', index_map)

## Обработка новых данных и обучение с их помощью классификатора
Для обучения классификатора создаем отдельный датафрейм, в котором данные из готовых вопросов mail.ru помечаем 0, а названия товаров 1. Все данные векторизуем с помощью HashingVectorizer и классифицируем с помощью LogisticRegression. Классификатор и векторизаторы сохраним в файлы, для последующего применения в функциях.

In [11]:
# Подготовим датафрейм с товарами
df = pd.read_csv('ProductsDataset.csv').drop(columns=['properties', 'image_links'])
df['descrirption'] = df['descrirption'].fillna(' ')
df['item'] = df['title'] + '. ' + df['descrirption']
#df['item'] = df['item'].apply(preprocess_txt)
#df['item'] = df['item'].apply(lambda x: [i for i in x if len(i) > 2])
df['preproc_title'] = df['title'].apply(preprocess_txt)
df['preproc_title'] = df['preproc_title'].apply(lambda x: ','.join(x))
df.to_csv('ready_ProductsDataset.csv', index=False)

In [12]:
#Подготовим список вопросов c mail.ru для обучения классификатора и сохраним в файл
questions = []
count = 0
with codecs.open("prepared_answers.txt", "r", "utf-8") as f:
    for line in tqdm(f):
        question = line.split("\t")[0]
        processed_question = ','.join(preprocess_txt(question))
        questions.append(processed_question)
        count += 1
        if count > 40000:
            break
            
for i, v in enumerate(questions):
    if not v:
        questions.pop(i)
        
pickle.dump(questions, open('questions_list_clf.pkl', 'wb'))

0it [00:00, ?it/s]

In [35]:
questions = pickle.load(open('questions_list_clf.pkl', 'rb'))
df = pd.read_csv('ready_ProductsDataset.csv')

In [43]:
df.dropna(inplace=True)

In [45]:
# Создадим датафрейм для обучения классификатора. Товарные запросы помечаем 1, все остальные 0
df_classif = pd.DataFrame(columns = ['query', 'mark'])
df_classif['query'] = questions[:35500]
df_classif['mark'] = 0

df_classif2 = pd.DataFrame(columns = ['query', 'mark'])
df_classif2['query'] = df['preproc_title']
df_classif2['mark'] = 1

df_clf = pd.concat([df_classif, df_classif2], ignore_index=True)

In [46]:
# Создадим и обучим классификатор, определяющий относится ли запрос к нашим товарам или просто "болталка"
# Векторизуем текст с помощью HashingVectorizer и потом преобразуем в обычный numpy вектор с TruncatedSVD
# Для классификации используем логистическую регрессию

from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix

train, test = train_test_split(df_clf, test_size = 0.2)

vectorizer_clf = HashingVectorizer(n_features=2**16, ngram_range=(1,1))
train_values = vectorizer_clf.fit_transform(train['query'])
pickle.dump(vectorizer_clf, open('hash_vectorizer.pkl', 'wb'))

svd_clf = TruncatedSVD(n_components=100, random_state=10)
train_X = svd_clf.fit_transform(train_values)
pickle.dump(svd_clf, open('svd_transformer.pkl', 'wb'))

test_values = vectorizer_clf.transform(test['query'])
test_X = svd_clf.transform(test_values)

train_y = train['mark'].values
test_y = test['mark'].values

model_clf = LogisticRegression(random_state=10).fit(train_X, train_y)
predicted_y = model_clf.predict(test_X)
pickle.dump(model_clf, open('model_clf.pkl', 'wb'))
print('Accuracy score: ', model_clf.score(test_X, test_y))
print('Confusion matrix:')
confusion_matrix(test_y, predicted_y)

Accuracy score:  0.8968818188217076
Confusion matrix:


array([[6974,  106],
       [1359, 5768]])

## Поиск подходящего товара по запросу
Для поиска подходящего товара(товаров) будем использовать также векторизатор HashingVectorizer, т.к. в продуктовых запросах мало контекстных слов, векторизовать с word2vec тут было бы менее эффективно. Сам поиск реализуем с помощью annoy. Обученные векторизаторы и индекс сохраняем в файлы.

In [15]:
# Поиск подходящего товара по запросу реализуем с помощью алгоритма Annoy.
# Для векторизации текста используем HashingVectorizer и TruncatedSVD

item_vectorizer = HashingVectorizer(n_features=2**13, ngram_range=(1,1))
item_values = item_vectorizer.fit_transform(df['preproc_title'])
pickle.dump(item_vectorizer, open('item_vectorizer.pkl', 'wb'))

item_svd = TruncatedSVD(n_components=200, random_state=10)
items = item_svd.fit_transform(item_values)
pickle.dump(item_svd, open('item_transformer.pkl', 'wb'))

item_index = annoy.AnnoyIndex(200 ,'angular')

for i, b in enumerate(items):
    item_index.add_item(i, b)
    
item_index.build(10)
item_index.save('goods_by_title_hash.ann')

True

## Реализация функции get_answer().
Логику работы чат-бота реализуем в функции get_answer. Она в себя включает 3 отдельных функции: классификатор, поиск товара и поиск ответа из разговорного жанра. На вход функция get_answer принимает запрос в формате строки и вторым аргументом, опционально, количество товаров для поиска. По умолчанию она находит один товар, либо дает один ответ на вопрос. При указании числа больше 1, функция вернет указаное число найденных товаров по запросу в формате датафрейма, либо один ответ на вопрос, если расценит запрос не как товарный.

In [48]:
# Подгружаем все наши обученые выше и сохраненные алгоритмы и индексы

# Для функции find_item
item_index = annoy.AnnoyIndex(200 ,'angular')
item_index.load('goods_by_title_hash.ann')
item_vectorizer = pickle.load(open('item_vectorizer.pkl', 'rb'))
item_svd = pickle.load(open('item_transformer.pkl', 'rb'))
df = pd.read_csv('ready_ProductsDataset.csv')

# Для функции classificator
vectorizer_clf = pickle.load(open('hash_vectorizer.pkl', 'rb'))
svd_clf = pickle.load(open('svd_transformer.pkl', 'rb'))
model_clf = pickle.load(open('model_clf.pkl', 'rb'))

In [49]:
def find_item(question, number):
    """
    Функция поиска товара из датасета с продуктами. Принимает вопрос в формате строки и кол-во
    товаров для поиска вторым аргументом. Возвращает строку при поиске одного товара, либо
    датафрейм при поиске более одного товара
    """

    preprocessed_question = [','.join(preprocess_txt(question))]
    vector = item_vectorizer.transform(preprocessed_question)
    vector = item_svd.transform(vector).reshape(-1)
    index = item_index.get_nns_by_vector(vector, number)
    if number==1:
        prod_id = df['product_id'].iloc[index].values
        prod_id = ''.join(list(prod_id))
        title = df['title'].iloc[index].values
        title = ''.join(list(title))
        return prod_id + " " + title
    if number > 1:
        return df[['product_id', 'title']].iloc[index]
    else:
        return 'Неверно указано количество товаров. Должно быть целое число больше 0.'

In [64]:
def classificator(text):
    """
    Функция, классифицирующая запрос на 'продуктовый' либо 'другой'.
    Принимает на вход вопрос в формате строки. 
    Возврщает 1 для продуктового запроса, либо 0 для всех остальных
    """
    
    vectorizer_clf = pickle.load(open('hash_vectorizer.pkl', 'rb'))
    svd_clf = pickle.load(open('svd_transformer.pkl', 'rb'))
    model_clf = pickle.load(open('model_clf.pkl', 'rb'))
    text = ','.join(preprocess_txt(text))
    text = vectorizer_clf.transform([text])
    text = svd_clf.transform(text)
    text_mark = model_clf.predict(text)
    return text_mark[0]

In [54]:
def speaker_answer(question):
    """
    Функция для поиска ответа из заготовленых с сайта mail.ru.
    На вход принимает вопрос в формате строки. Возвращает ответ в формате строки.
    В работе использует модель и индекс обученные и сохраненные уроком ранее, и загруженые в начале блокнота
    """
    
    preprocessed_question = preprocess_txt(question)
    n_w2v = 0
    vector = np.zeros(50)
    for word in preprocessed_question:
        if word in model_speaker.wv:
            vector += model_speaker.wv[word]
            n_w2v += 1
    if n_w2v > 0:
        vector = vector / n_w2v
    answer_index = index_speaker.get_nns_by_vector(vector, 1)
    return index_map[answer_index[0]]

In [52]:
def get_answer(question, number=1):
    """
    Функция, дающая ответ по запросу. Принимает запрос в формате строки и число.
    В зависимости от классификации запроса возвращает: ответ в формате строки, для непродуктового запроса;
    id товара и его название в формате строки, для продуктового запроса с числом 1;
    датафрейм с товарами при продуктовом запросе и числе больше 1.
    Если не указать число при запросе, по умолчанию стоит 1.
    Если запрос классифицируется как непродуктовый, то указанное число не имеет значения.
    """
    mark = classificator(question)
    if mark==1:
        return find_item(question, number)
    else:
        return speaker_answer(question)

In [65]:
get_answer('как дела?', 1)

[[ 5.11050695e-05  2.72745200e-04  1.04215967e-04 -3.73541623e-05
   1.38280136e-04  3.90471444e-04  4.23034361e-05  2.52482067e-04
  -6.32067193e-04  6.51667594e-04  1.30398731e-03 -1.61155261e-03
  -2.13432318e-03  9.53850638e-03 -9.55484525e-05 -2.31221734e-05
  -6.87750259e-04 -1.26184370e-03  1.89653528e-04 -1.89369278e-04
   8.20258371e-04  1.12288399e-04  1.13469959e-03  4.13062088e-04
   8.87835399e-04  1.12627823e-03  1.38333537e-03  2.83293947e-02
  -4.40114796e-03  5.40248737e-03  2.70266601e-04 -6.85659030e-03
  -9.11316664e-06  1.55768666e-03 -1.16994497e-02 -1.03249204e-03
   1.77475407e-03 -1.70051091e-03  1.00702291e-03  1.37115899e-03
  -2.25566854e-04  9.45436173e-05 -1.10175061e-03  1.61184864e-03
  -3.71888367e-03 -4.83066330e-03  3.38370209e-03 -2.63570136e-03
   7.69141416e-03  1.79373270e-03 -3.69529650e-04 -6.59886118e-04
   6.48609075e-04  7.42161464e-03 -2.98408806e-03  1.69286056e-03
   1.63019258e-03  4.17414911e-03  2.56682893e-03 -2.16102308e-04
   4.11689

'хуже уже некуда. \n'